In [1]:
from utils import *
import os
import pandas as pd
import sklearn
from sklearn import *
import os
import errno
import pickle

# Simple model (Count Vectorizer)

In [2]:
path_folder_quora = "./Datasets/QuoraQuestionPairs/"

In [3]:
train_df = pd.read_csv(os.path.join(path_folder_quora, "quora_train_data.csv"))
A_df, te_df = sklearn.model_selection.train_test_split(train_df,test_size=0.05,random_state=123)
tr_df, va_df = sklearn.model_selection.train_test_split(A_df,test_size=0.05,random_state=123)
print('tr_df.shape=',tr_df.shape)
print('va_df.shape=',va_df.shape)
print('te_df.shape=',te_df.shape)

tr_df.shape= (291897, 6)
va_df.shape= (15363, 6)
te_df.shape= (16172, 6)


In [4]:
tr_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
61482,125898,203030,203031,Is Java or C++ or C the most popular language ...,How do I develop a software which will have a ...,0
131546,36249,66113,66114,How do you convert direct speech into reported...,I feel weak at spoken English. I have sentence...,0
22927,199864,301469,301470,Where can I buy used wine barrels?,Where can you buy used wine barrels?,1
183520,277339,17728,138400,What was the best day of your life? (Excluding...,What is the Best Day of your life till date?,1
67694,392907,525647,525648,How is web-work.in works?,How do I get web designing work?,0


In [5]:
q1_train =  cast_list_as_strings(list(tr_df["question1"]))
q2_train =  cast_list_as_strings(list(tr_df["question2"]))
q1_validation =  cast_list_as_strings(list(va_df["question1"]))
q2_validation =  cast_list_as_strings(list(va_df["question2"]))

In [6]:
q1_train[0], q2_train[0]

('Is Java or C++ or C the most popular language amongst startups for backend development?',
 'How do I develop a software which will have a Java GUI and a C++ or C backend?')

In [7]:
all_train_questions = q1_train + q2_train

count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
count_vectorizer.fit(all_train_questions)

CountVectorizer()

In [8]:
X_tr_q1q2 = get_features_from_df(tr_df, count_vectorizer)

X_tr_q1q2.shape, tr_df.shape

((291897, 149650), (291897, 6))

In [9]:
logistic = sklearn.linear_model.LogisticRegression(solver="liblinear", random_state=123)
y_train = tr_df["is_duplicate"].values
logistic.fit(X_tr_q1q2, y_train)

LogisticRegression(random_state=123, solver='liblinear')

In [10]:
mistake_indices_tr, predictions_tr = get_mistakes(logistic, X_tr_q1q2,  y_train)

print_mistake_k(10, tr_df, mistake_indices_tr, predictions_tr)

What is a good website for free ebooks?
What are the sites to download free eBooks?
true class: 1
prediction: 0


In [11]:
# check if directory exists
if not os.path.exists('model_artifacts'):
    try:
        os.makedirs('model_artifacts')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

# open file descriptors for writing
countvectorizer_file = open('model_artifacts/count_vectorizer.pkl', 'wb')
logistic_file = open('model_artifacts/logistic.pkl', 'wb')

# serialize and write objects to files
pickle.dump(count_vectorizer, countvectorizer_file)
pickle.dump(logistic, logistic_file)

# close file descriptors
countvectorizer_file.close()
logistic_file.close()

In [12]:
# w1vec = word2vec.transform(w1)
# len(w1vec) = 256

# w2vec = word2vec.transform(w2)
# len(w2vec) = 256

# EmbA  = w1vec + ... + wnvec -> 256
# EmbB  = v1vec + ... + vnvec -> 256
# concat(256, 256)            -> 512
# word2vec feature: d(EmbA, EmbB)
# X_q1q2
# X_q1q2_fMaria_fPedro_fAdrian -> XGBoost / Logistic

# Improved model